### DeltaTopic Basics

In [1]:
!python3 -m pip install DeltaTopic

In [2]:
import os
import pandas as pd
import numpy as np
import DeltaTopic.datasets as dt
from DeltaTopic.nn.util import setup_anndata
from DeltaTopic.nn.modelhub import DeltaTopic

#### Data Prepraration

In [3]:
adata = dt.toy_data()
setup_anndata(adata, layer="counts", unspliced_obsm_key = "unspliced_expression")

#### Model Training

In [4]:
model = DeltaTopic(adata, n_latent = 8)
model.train(10)

SavePATH = "./my_model_DIR/"
model_id = "my_model"
model.save(os.path.join(SavePATH, model_id), overwrite=True, save_anndata=False)
print(f"Model saved at:", os.path.join(SavePATH, model_id))

/home/BCCRC.CA/yzhang/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:467: LightningDeprecationWarning: Setting `Trainer(gpus=[0])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[0])` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name   | Type              | Params
---------------------------------------------
0 | module | DeltaTopic_module | 1

Epoch 9: 100%|██████████| 71/71 [00:01<00:00, 62.20it/s, loss=5.32e+04, v_num=1]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 71/71 [00:01<00:00, 62.04it/s, loss=5.32e+04, v_num=1]
Model saved at: ./my_model_DIR/my_model


#### Analysis
##### Estimate topic proportions

In [5]:
topics_np = model.get_latent_representation() # save latent topic proportions
pd.DataFrame(topics_np).to_csv(os.path.join('topics.csv'))
topics_np.shape

Deterministic: True, output_softmax_z: True


(10000, 8)

##### Compute the expected weight from spike and slab parameters

In [6]:
# save spike and slab parameters from the model
model.get_parameters(save_dir = os.path.join(SavePATH, model_id)) 

spike_logit_rho = np.loadtxt(os.path.join(SavePATH, model_id, "model_parameters", "spike_logit_rho.txt"))
slab_mean_rho = np.loadtxt(os.path.join(SavePATH, model_id, "model_parameters", "spike_logit_rho.txt"))
pip_rho = 1/np.exp(-spike_logit_rho)    
weight_rho = slab_mean_rho * pip_rho

spike_logit_delta = np.loadtxt(os.path.join(SavePATH, model_id, "model_parameters", "spike_logit_delta.txt"))
slab_mean_delta = np.loadtxt(os.path.join(SavePATH, model_id, "model_parameters", "spike_logit_delta.txt"))
pip_delta = 1/np.exp(-spike_logit_delta)
weight_delta = slab_mean_delta * pip_delta

print(weight_rho.shape)
print(weight_delta.shape)

(8, 5000)
(8, 5000)
